In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import os
import scipy.ndimage as nd
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt
import skimage.measure as sk
from utils import utils
plt.style.use('ggplot')
from sklearn import metrics

In [3]:
from torch.autograd import Variable
from torchvision import datasets
from torchvision import transforms as tfs
from torch.utils import data
import torch
import PIL.ImageOps
import skimage
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

In [4]:
import torch.optim as optim
from torch import nn
from models.classifierResnet import _C
import visdom
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import TopKCategoricalAccuracy, Loss
from ignite.handlers import ModelCheckpoint
import torch.nn.functional as F
import pickle

In [5]:
available_device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
%%timeit
#### Load dataset

transformations = [tfs.RandomAffine(0., scale=(0.5, 1.), fillcolor=0),
                   tfs.Resize((128, 128)),
                   #tfs.Grayscale(1),
                   tfs.Lambda(lambda x: PIL.ImageOps.invert(x))]

imagenet_data = datasets.ImageFolder('data/perfiles_CATA/png_clasificados/',
                                     transform=tfs.Compose(transformations))

data_loader = data.DataLoader(imagenet_data, batch_size=4, shuffle=False)

#### Process type of vessels for classification

df = utils.create_df_from_files(path='data/perfiles_CATA/clases/')

#### Build and train classifier

imagenet_data = datasets.ImageFolder('data/perfiles_CATA/png_clasificados/',
                                     transform=tfs.Compose([tfs.RandomHorizontalFlip(p=0.7),
                                                           tfs.RandomAffine(0, scale=(0.7, 1.), fillcolor=0),
                                                           tfs.Resize((128, 128)),
                                                           tfs.ToTensor()]))
                                                           

splits_len = round(len(imagenet_data.samples)*0.2), round(len(imagenet_data.samples)*0.1), round(len(imagenet_data.samples)*0.7)

splits_len = splits_len if np.sum(splits_len) == len(imagenet_data.samples) else splits_len[0]-1, splits_len[1], splits_len[2] +2

#### Split

#All
splits = utils.random_split(imagenet_data, splits_len)

file_pi2 = open('data_pickle/indice_0.pickle', 'rb') 
splits[0].indices = pickle.load(file_pi2)

file_pi2 = open('data_pickle/indice_1.pickle', 'rb') 
splits[2].indices = pickle.load(file_pi2)

train_loader = data.DataLoader(splits[2], batch_size=32, shuffle=False)
test_loader = data.DataLoader(splits[0], batch_size=32, shuffle=False)

C =_C(input_h_w=128)
C = C.to(available_device)

checkpoint = torch.load("models/checkpoint_data_resnet/cnn_vessels_model_100.pth")
C.load_state_dict(checkpoint)

y_real_test, X_test = utils.iterations_test_partial(C, test_loader, available_device)
y_real_train, X_train = utils.iterations_test_partial(C, train_loader, available_device)

3.73 s ± 244 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
#### Load dataset

transformations = [tfs.RandomAffine(0., scale=(0.5, 1.), fillcolor=0),
                   tfs.Resize((128, 128)),
                   #tfs.Grayscale(1),
                   tfs.Lambda(lambda x: PIL.ImageOps.invert(x))]

imagenet_data = datasets.ImageFolder('data/perfiles_CATA/png_clasificados/',
                                     transform=tfs.Compose(transformations))

data_loader = data.DataLoader(imagenet_data, batch_size=4, shuffle=False)

#### Process type of vessels for classification

df = utils.create_df_from_files(path='data/perfiles_CATA/clases/')

#### Build and train classifier

imagenet_data = datasets.ImageFolder('data/perfiles_CATA/png_clasificados/',
                                     transform=tfs.Compose([tfs.RandomHorizontalFlip(p=0.7),
                                                           tfs.RandomAffine(0, scale=(0.7, 1.), fillcolor=0),
                                                           tfs.Resize((128, 128)),
                                                           tfs.ToTensor()]))
                                                           

splits_len = round(len(imagenet_data.samples)*0.2), round(len(imagenet_data.samples)*0.1), round(len(imagenet_data.samples)*0.7)

splits_len = splits_len if np.sum(splits_len) == len(imagenet_data.samples) else splits_len[0]-1, splits_len[1], splits_len[2] +2

#### Split

#All
splits = utils.random_split(imagenet_data, splits_len)

file_pi2 = open('data_pickle/indice_0.pickle', 'rb') 
splits[0].indices = pickle.load(file_pi2)

file_pi2 = open('data_pickle/indice_1.pickle', 'rb') 
splits[2].indices = pickle.load(file_pi2)

train_loader = data.DataLoader(splits[2], batch_size=32, shuffle=False)
test_loader = data.DataLoader(splits[0], batch_size=32, shuffle=False)

C =_C(input_h_w=128)
C = C.to(available_device)

checkpoint = torch.load("models/checkpoint_data_resnet/cnn_vessels_model_100.pth")
C.load_state_dict(checkpoint)

y_real_test, X_test = utils.iterations_test_partial(C, test_loader, available_device)
y_real_train, X_train = utils.iterations_test_partial(C, train_loader, available_device)

# SVM

In [8]:
from sklearn.svm import SVC

In [9]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_real_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
%%timeit
clf = SVC(gamma='auto')
clf.fit(X_train, y_real_train)

149 ms ± 22.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
y_pred_test = clf.predict(X_test)

44.1 ms ± 95 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# RandomForestClassifier

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
clf = RandomForestClassifier(n_estimators=500,max_depth=4)
clf.fit(X_train, y_real_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
%%timeit
clf = RandomForestClassifier(n_estimators=500,max_depth=4)
clf.fit(X_train, y_real_train)

1.9 s ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
y_pred_test = clf.predict(X_test)

32.4 ms ± 74 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
